## Modelling

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("cleaned_data.csv")
df.head()

,Title,Genre,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired
0,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",TV,['Bandai Visual'],['Sunrise'],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999"
1,Cowboy Bebop Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001"
2,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",TV,['Victor Entertainment'],['Madhouse'],8.31,197451.0,158.0,372709.0,26.0,Manga,"Apr 1, 1998 to Sep 30, 1998"
3,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",TV,['Bandai Visual'],['Sunrise'],7.34,31875.0,1278.0,74889.0,26.0,Original,"Jul 2, 2002 to Dec 24, 2002"
4,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",TV,NaN,['Toei Animation'],7.04,4757.0,3968.0,11247.0,52.0,Manga,"Sep 30, 2004 to Sep 29, 2005"


In [5]:
from sklearn.feature_extraction.text import TfidVectorizer
vectorizer = TfidVectorizer(ngram_range = (1,2))

ModuleNotFoundError: No module named 'sklearn'